# Pattern Matching in a Social Network

In [ ]:
import swat
import pandas as pd

In [ ]:
s = swat.CAS(host, port, username, password)

In [ ]:
s.loadActionSet('Network')

## Input data

In [ ]:
nodes = [['Matt', 'Person'],
         ['Rob', 'Person'],
         ['Chuck', 'Person'],
         ['Stephen', 'Person'],
         ['Manoj', 'Person'],
         ['Bryan', 'Person'],
         ['Jack', 'Person'],
         ['Raleigh', 'City'],
         ['Philadelphia', 'City'],
         ['Charlotte', 'City'],
         ['The Pit Authentic', 'Restaurant', 'BBQ'],
         ['Red Hot Blue', 'Restaurant', 'BBQ'],
         ['JimmyJs', 'Restaurant', 'BBQ'],
         ['Second Empire', 'Restaurant', 'American'],
         ['Cafe Luna', 'Restaurant', 'Italian'],
         ['Vivo Rist', 'Restaurant', 'Italian'],
         ['Moonlight', 'Restaurant', 'Itlian'],
         ['Dumplings', 'Restaurant', 'Chinese']]

nodes = pd.DataFrame(nodes, columns = ['node', 'type', 'subtype'])
nodes['label'] = nodes['node']

In [ ]:
links = [['Matt', 'Rob', 'friends'],
         ['Rob', 'Matt', 'friends'],
         ['Matt', 'Chuck', 'friends'],
         ['Chuck', 'Matt', 'friends'],
         ['Chuck', 'Rob', 'friends'],
         ['Rob', 'Chuck', 'friends'],
         ['Jack', 'Rob', 'friends'],
         ['Rob', 'Jack', 'friends'],
         ['Matt', 'Stephen', 'friends'],
         ['Stephen', 'Matt', 'friends'],
         ['Matt', 'Manoj', 'friends'],
         ['Manoj', 'Matt', 'friends'],
         ['Matt', 'Bryan', 'friends'],
         ['Bryan', 'Matt', 'friends'],
         ['Matt', 'Jack', 'friends'],
         ['Jack', 'Matt', 'friends'],
         ['Matt', 'Philadelphia', 'lives in'],
         ['Stephen', 'Philadelphia', 'lives in'],
         ['Stephen', 'JimmyJs', 'likes', 7],
         ['Stephen', 'Cafe Luna', 'likes', 8],
         ['Rob', 'Raleigh', 'lives in'],
         ['Chuck', 'Raleigh', 'lives in'],
         ['Manoj', 'Raleigh', 'lives in'],
         ['Jack', 'Raleigh', 'lives in'],
         ['Bryan', 'Charlotte', 'lives in'],
         ['Rob', 'The Pit Authentic', 'likes', 7],
         ['Jack', 'Red Hot Blue', 'likes', 9],
         ['Chuck', 'The Pit Authentic', 'likes', 8],
         ['Chuck', 'Cafe Luna', 'likes', 6],
         ['Chuck', 'Second Empire', 'likes', 7],
         ['Jack', 'Vivo Rist ', 'likes', 8],
         ['Manoj', 'Dumplings', 'likes', 6],
         ['Bryan', 'Red Hot Blue', 'likes', 9],
         ['Bryan', 'Vivo Rist', 'likes', 6],
         ['Rob', 'Moonlight', 'likes', 10]]
links = pd.DataFrame(links, columns = ['from', 'to', 'connection', 'rating'])

In [ ]:
s.upload(nodes, casout = {'name':'nodes', 'replace':True})
s.upload(links, casout = {'name':'links', 'replace':True})

## Query: "friends of Matt who like barbecue restaurants"

In [ ]:
nodesQuery = [['Matt', 'Matt', 'Person', ''],
              ['X', '', 'Person', ''],
              ['BBQ', '', 'Restaurant', 'BBQ']]
linksQuery = [['Matt', 'X', 'friends'],
              ['X', 'Matt', 'friends'],
              ['X', 'BBQ', 'likes']]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node', 'label', 'type', 'subtype'])
linksQuery = pd.DataFrame(linksQuery, columns = ['from', 'to', 'connection'])

In [ ]:
s.upload(nodesQuery, casout = {'name':'nodesQuery', 'replace':True})
s.upload(linksQuery, casout = {'name':'linksQuery', 'replace':True})

In [ ]:
s.network.patternMatch(
    direction      = "directed",
    links          = {"name":"links"},
    nodes          = {"name":"nodes"},
    linksQuery     = {"name":"linksQuery"},
    nodesQuery     = {"name":"nodesQuery"},
    nodesVar       = {"vars":("label", "type", "subtype")},
    linksVar       = {"vars":"connection"},
    nodesQueryVar  = {"vars":("label", "type", "subtype")},
    linksQueryVar  = {"vars":"connection"},
    outNodes       = {"name":"OutNodes", "replace":True},
    outMatchNodes  = {"name":"OutMatchNodes", "replace":True},
    outMatchLinks  = {"name":"OutMatchLinks", "replace":True})

In [ ]:
print(s.CASTable("OutMatchNodes").to_frame())
print(s.CASTable("OutMatchLinks").to_frame())

## Query: "friends of Matt who like barbecue restaurants and live in Raleigh"

In [ ]:
nodesQuery = [['Matt', 'Matt', 'Person'],
              ['X', '', 'Person'],
              ['Raleigh', 'Raleigh', 'City'],
              ['BBQ', '', 'Restaurant', 'BBQ']]
linksQuery = [['Matt', 'X', 'friends'],
              ['X', 'Matt', 'friends'],
              ['X', 'Raleigh', 'lives in'],
              ['X', 'BBQ', 'likes']]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node', 'label', 'type', 'subtype'])
linksQuery = pd.DataFrame(linksQuery, columns = ['from', 'to', 'connection'])

In [ ]:
s.upload(nodesQuery, casout = {'name':'nodesQuery', 'replace':True})
s.upload(linksQuery, casout = {'name':'linksQuery', 'replace':True})

In [ ]:
s.network.patternMatch(
    direction      = "directed",
    links          = {"name":"links"},
    nodes          = {"name":"nodes"},
    linksQuery     = {"name":"linksQuery"},
    nodesQuery     = {"name":"nodesQuery"},
    nodesVar       = {"vars":("label", "type", "subtype")},
    linksVar       = {"vars":"connection"},
    nodesQueryVar  = {"vars":("label", "type", "subtype")},
    linksQueryVar  = {"vars":"connection"},
    outNodes       = {"name":"OutNodes", "replace":True},
    outMatchNodes  = {"name":"OutMatchNodes", "replace":True},
    outMatchLinks  = {"name":"OutMatchLinks", "replace":True}
)

In [ ]:
print(s.CASTable("OutMatchNodes").to_frame())
print(s.CASTable("OutMatchLinks").to_frame())

## Query: "friends of Matt who like barbecue restaurants, give the restaurant a rating of 9 or higher, and live in Raleigh"

In [ ]:
s.sessionProp.setSessOpt(cmplib = "casuser.myRoutines")
s.loadActionSet('fcmpact')

In [ ]:
s.addRoutines(
    routineCode = '''
    function myLinkFilter(connectionQ $, rating); 
        if (connectionQ = 'likes') then return (rating >= 9); 
        else return (1); 
    endsub;''',
    package   = "myPackage",
    saveTable = True,
    funcTable = {"name": "myRoutines", "replace": True}
)    

In [ ]:
s.network.patternMatch(
    direction      = "directed",
    links          = {"name":"links"},
    nodes          = {"name":"nodes"},
    linksQuery     = {"name":"linksQuery"},
    nodesQuery     = {"name":"nodesQuery"},
    nodesVar       = {"vars":("label", "type", "subtype")},
    linksVar       = {"vars":("connection", "rating")},
    nodesQueryVar  = {"vars":("label", "type", "subtype")},
    linksQueryVar  = {"vars":"connection"},
    outNodes       = {"name":"OutNodes", "replace":True},
    outMatchNodes  = {"name":"OutMatchNodes", "replace":True},
    outMatchLinks  = {"name":"OutMatchLinks", "replace":True},  
    linkFilter     = "myLinkFilter"
)

In [ ]:
print(s.CASTable("OutMatchNodes").to_frame())
print(s.CASTable("OutMatchLinks").to_frame())

## Query: "a pair of friends of Matt who like the same barbecue restaurant, live in Raleigh, and are friends of each other"

In [ ]:
nodesQuery = [['Matt', 'Matt', 'Person'],
              ['X', '', 'Person'],
              ['Y', '', 'Person'],
              ['Raleigh', 'Raleigh', 'City'],
              ['BBQ', '', 'Restaurant', 'BBQ']]
linksQuery = [['Matt', 'X', 'friends'],
              ['X', 'Matt', 'friends'],
              ['Matt', 'Y', 'friends'],
              ['Y', 'Matt', 'friends'],
              ['X', 'Raleigh', 'lives in'],
              ['Y', 'Raleigh', 'lives in'],
              ['X', 'BBQ', 'likes'],
              ['Y', 'BBQ', 'likes'],
              ['X', 'Y', 'friends'],
              ['Y', 'X', 'friends']]
nodesQuery = pd.DataFrame(nodesQuery, columns = ['node', 'label', 'type', 'subtype'])
linksQuery = pd.DataFrame(linksQuery, columns = ['from', 'to', 'connection'])

In [ ]:
s.upload(nodesQuery, casout = {'name':'cas_nodesQuery', 'replace':True})
s.upload(linksQuery, casout = {'name':'cas_linksQuery', 'replace':True})

In [ ]:
s.network.patternMatch(
    direction      = "directed",
    links          = {"name":"links"},
    nodes          = {"name":"nodes"},
    linksQuery     = {"name":"linksQuery"},
    nodesQuery     = {"name":"nodesQuery"},
    nodesVar       = {"vars":("label", "type", "subtype")},
    linksVar       = {"vars":"connection"},
    nodesQueryVar  = {"vars":("label", "type", "subtype")},
    linksQueryVar  = {"vars":"connection"},
    outNodes       = {"name":"OutNodes", "replace":True},
    outMatchNodes  = {"name":"OutMatchNodes", "replace":True},
    outMatchLinks  = {"name":"OutMatchLinks", "replace":True}
)

In [ ]:
print(s.CASTable("OutMatchNodes").to_frame())
print(s.CASTable("OutMatchLinks").to_frame())